Code for **"Phase Retrieval"** (testing wrong phase)

# Import libs

In [ ]:
from __future__ import print_function
import matplotlib.pyplot as plt
%matplotlib inline

import os
import numpy as np
from models.resnet import ResNet
from models.unet import UNet
from models.skip import skip
import torch
import torch.optim
from utils.inpainting_utils import *
from utils.utils import *

# Misc. Setup

In [ ]:
# Enable GPU usage
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark =True
dtype = torch.cuda.FloatTensor

# Change cwd
directory = "/home/justin/Github/deep-image-prior/"
os.chdir(directory)

# Misc.
PLOT = True
imsize = -1
dim_div_by = 64

# Load Image and Measurement

In [ ]:
# Set Image
img_type = 'grayscale'
img_path  = 'data/inpainting/vase.png'
_, img_np = get_image(img_path, imsize, img_type)
img_var = np_to_torch(img_np, True).type(dtype)

# Print Image
plt.imshow(img_np[0], cmap = 'gray')
plt.show()

# Generate Gaussian Matrix (A)
m = int(float(np.shape(img_np)[1])/2.0)
n = np.shape(img_np)[1]
mu = 0.0
sigma = 0.25

A = mu + sigma*np.random.randn(m,1,n)
A_var = np_to_torch(A, False).type(dtype)

# Set Measurement
measurement = np.tensordot(A,img_np,2)

# Flip phase in ~n places (sign)
N = np.prod(measurement.shape)
num_flips = N/8
p0 = num_flips/N; p1 = 1-p0
flip_choices = np.random.choice([-1, 1], size=measurement.shape, p=[p0, p1])
ones = np.ones(measurement.shape)
flip_matrix = np.multiply(flip_choices,ones)
measurement = np.multiply(flip_matrix,measurement)

unique, counts = np.unique(flip_matrix, return_counts=True)
print(dict(zip(unique, counts)))

# # Randomize Phase (sign)
# measurement = np.absolute(measurement)
# random_sign = np.random.choice([-1,1],size=measurement.shape)
# measurement = np.multiply(random_sign,measurement)

# Save Measurement
measurement_var = np_to_torch(measurement, False).type(dtype)

# Setup

In [ ]:
# Optimization Setup
pad = 'reflection' # 'zero'
OPT_OVER = 'net'
OPTIMIZER = 'adam'
NET_TYPE = 'skip_depth6' # one of skip_depth4|skip_depth2|UNET|ResNet


#--------------------------
# Network Arch. Setup
#--------------------
INPUT = 'meshgrid'
input_depth = 2                   # depth of input noise
output_depth = 1
LR = 0.01                         # learning rate
num_iter = 1000
param_noise = False               # add noise to net params during optimization 
show_every = 100
figsize = 5
reg_noise_std = 0.03              # add noise to net input during optimization

num1 = 5 # 5
num2 = 3 # 3
num3 = 128 # 128

net = skip(input_depth, output_depth, 
           num_channels_down = [num3] * num1,
           num_channels_up   = [num3] * num1,
           num_channels_skip = [0] * num1,  
           upsample_mode='nearest', filter_skip_size=1, filter_size_up=num2, filter_size_down=num2,
           need_sigmoid=True, need_bias=True, pad=pad, act_fun='LeakyReLU').type(dtype)


net = net.type(dtype)
img_dim = img_np.shape[1:]
net_input = get_noise(input_depth, INPUT, img_dim).type(dtype)
#--------------------------

# Compute number of parameters
s  = sum(np.prod(list(p.size())) for p in net.parameters())
print ('Number of params: %d' % s)

# Loss
mse = torch.nn.MSELoss().type(dtype)

# Main loop

In [ ]:
i = 0

#----------------------------------
def dip_function():

    global i
    
    # Add noise to network parameters / network input
    if param_noise:
        for n in [x for x in net.parameters() if len(x.size()) == 4]:
            n = n + n.detach().clone().normal_() * n.std() / 50
            
    net_input = net_input_saved
    if reg_noise_std > 0:
        net_input = net_input_saved + (noise.normal_() * reg_noise_std)
        
    net_input_np = torch_to_np(net_input)
    
    # Output of network
    out = net(net_input)   
    out_np = torch_to_np(out)
    
    # Calculate measurement estimate |A*net_out|
    new_shape = tuple(out.shape[1:])    # eliminate unnecessary dimension
    out = out.view(new_shape)           # must reshape before tensordot
    measurement_hat = tensordot_pytorch(A_var,out,axes=2)
    
    # Define Loss (avg. |A*net_out-B**A*image|l2)
    total_loss = mse(measurement_hat, measurement_var)
    total_loss.backward()
    
    # Print Progress
    print ('Iteration %05d    Loss %f' % (i, total_loss.item()), '\r', end='')
    if  PLOT and i % show_every == 0:
        
        dir_ = "./plots/wrong_phase/"
        name = "reconst_iter: "+str(i)+".png"
        plot_image_grid([np.clip(out_np, 0, 1)], factor=figsize,
                        nrow=1, save=True, directory=dir_, filename=name)
        
    i += 1
    return total_loss

#----------------------------------

# Final Setup
net_input_saved = net_input.detach().clone()
noise = net_input.detach().clone()

# Optimize
p = get_params(OPT_OVER, net, net_input)
losses = optimize(OPTIMIZER, p, dip_function, LR, num_iter)

In [ ]:
# Plot Final Estimate
out_np = torch_to_np(net(net_input))
plot_image_grid([out_np], factor=5);

# Plot Loss vs Iteration
plt.plot(np.arange(1,num_iter+1),losses)
directory = "./plots/wrong_phase/"
plt.savefig(directory+'loss vs iteration')
plt.show()

In [ ]:
# Save Data

import pickle
my_object = losses
filename = directory + "losses.obj"

with open(filename, 'wb') as fp:
    pickle.dump(my_object, fp)

In [ ]:
# Plot Measurements

print(measurement_var.shape)
plt.imshow(measurement_var, cmap = 'gray')
plt.show()

In [ ]:
# Analyze why phase retrieval (approx. =) lin. comp. sensing

print(measurement_var)

In [ ]:
# Load Previous Run
filename = "./plots/lin_comp_sense/" + "losses.obj"
with open (filename, 'rb') as fp:
    losses_normal = pickle.load(fp)
losses_normal = losses_normal[0:num_iter]
losses_wrong = losses

# Plot Losses simultaneously

plt.plot(np.arange(1,num_iter+1),losses_normal)
plt.plot(np.arange(1,num_iter+1),losses_wrong)

import matplotlib.patches as mpatches
directory = "./plots/"
orange_patch = mpatches.Patch(color='blue', label='normal_phase')
blue_patch = mpatches.Patch(color='orange', label='wrong_phase')
plt.legend(handles=[orange_patch,blue_patch])
plt.xlabel('iteration')
plt.ylabel('avg. |A*net_out-B**(A*image)|l2')
plt.savefig(directory+'loss vs iteration(wrong_phase)')

plt.show()